<a href="https://colab.research.google.com/github/N-avi-N/BERT_Notebooks/blob/main/Sentiment_Analysis_with_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data and EDA

In [3]:
!pip install transformers

     |████████████████████████████████| 2.8 MB 5.1 MB/s 
     |████████████████████████████████| 636 kB 68.5 MB/s 
     |████████████████████████████████| 3.3 MB 56.3 MB/s 
     |████████████████████████████████| 895 kB 60.5 MB/s 
     |████████████████████████████████| 50 kB 7.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/smileannotationsfinal.csv',
                 names =['id', 'text', 'category'])
df.set_index('id', inplace=True)

In [6]:
df['text'].iloc[0]

'@aandraous @britishmuseum @AndrewsAntonio Merci pour le partage! @openwinemap'

In [7]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [8]:
df.shape

(3085, 2)

In [9]:
df['category'].value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

Removing enteries with 2 categories(sepertated by '|' and 'nocode' enteries)

In [10]:
df = df.loc[~df['category'].str.contains('\|')]

In [11]:
df = df.loc[df['category']!='nocode']

In [12]:
df['category'].value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [13]:
possible_labels = df['category'].unique()

In [14]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label] = index

label_dict

{'angry': 2,
 'disgust': 3,
 'happy': 0,
 'not-relevant': 1,
 'sad': 4,
 'surprise': 5}

In [15]:
df['label'] = df['category'].replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


# Train/Validation split

the data split method is too complicated and can be simplified

In [16]:
from sklearn.model_selection import train_test_split

split index and then pass it to loc to split dataset

In [17]:
x_train, x_val, y_train, y_val = train_test_split(df.index.values,
                                                   df.label.values,
                                                   test_size = 0.15,
                                                   random_state = 21,
                                                   stratify = df.label.values)

In [18]:
df['data_type'] = ['not_set']*df.shape[0]

In [19]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [20]:
df.loc[x_train, 'data_type'] = 'train'
df.loc[x_val, 'data_type'] = 'val'

In [21]:
df.groupby(['category','label','data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

# Loading Tokenizer and Encoding data

In [22]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [23]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [24]:
encoded_data_train = tokenizer.batch_encode_plus(
                              df[df['data_type']=='train'].text.values,
                              add_special_tokens=True,
                              return_attention_mask=True,
                              padding='max_length',
                              max_length=256,
                              return_tensors='pt')

In [25]:
encoded_data_val = tokenizer.batch_encode_plus(
                              df[df['data_type']=='val'].text.values,
                              add_special_tokens=True,
                              return_attention_mask=True,
                              padding='max_length',
                              max_length=256,
                              return_tensors='pt')

get processed tokens, attention masks and labels from tokenizer

In [26]:
input_ids_train = encoded_data_train['input_ids']
attention_mask_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df['data_type']=='train'].label.values)

In [27]:
input_ids_val = encoded_data_val['input_ids']
attention_mask_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df['data_type']=='val'].label.values)

combine input_ids, attention_masks and labels 

In [28]:
dataset_train = TensorDataset(input_ids_train, attention_mask_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_mask_val, labels_val)

In [29]:
len(dataset_train), len(dataset_val)

(1258, 223)

# Create Train and Validation dataloaders

In [30]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [31]:
batch_size = 16

dataloader_train = DataLoader(dataset_train,
                              sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)

dataloader_val = DataLoader(dataset_val,
                            sampler=RandomSampler(dataset_val),
                            batch_size=batch_size)

# Setting up BERT pretrained model

In [32]:
from transformers import BertForSequenceClassification

In [33]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
                                                      num_labels = len(label_dict),
                                                      output_attentions = False,
                                                      output_hidden_states = False)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# Setting up Scheduler and Optimizer

In [34]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [35]:
optimizer = AdamW(model.parameters(),
                  lr = 1e-5,
                  eps = 1e-8)

In [36]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(optimizer=optimizer,
                                    num_warmup_steps=0,
                                    num_training_steps=len(dataloader_train)*epochs)

# Defining Performance metrics

In [37]:
import numpy as np

In [38]:
from sklearn.metrics import f1_score

In [39]:
def f1_score_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  
  return f1_score(labels_flat, preds_flat, average='weighted')

In [53]:
def accuracy_per_class(preds, labels):

  labels_dict_inverse = {v: k for k, v in label_dict.items()}

  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class:{labels_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

# Creating training loop

Approach adapted from an older version of HuggingFace's `run_glue.py` script. Accessible [here](https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128).

In [41]:
import random

seed_val = 21
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [42]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
model.to(device)

print(device)

cuda


Define evaluate funtion

In [43]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


Define train loop

In [48]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                       desc='Epoch {:1d}'.format(epoch),
                       leave=False,
                       disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            'labels':         batch[2],
        }
        
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total+=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/
                                                len(batch))})
        
    #torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 score(weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/79 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.34128017865027055
Validation loss: 0.5409678093024662
F1 score(weighted): 0.8172205175291773


Epoch 2:   0%|          | 0/79 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.26227311573073836
Validation loss: 0.49667652589934214
F1 score(weighted): 0.8493638916967464


...


Epoch 8
Training loss: 0.10380282373273675
Validation loss: 0.5829512965865433
F1 score(weighted): 0.8444446443867011


Epoch 9:   0%|          | 0/79 [00:00<?, ?it/s]


Epoch 9
Training loss: 0.10013626746927635
Validation loss: 0.5848157559188881
F1 score(weighted): 0.8444446443867011


Epoch 10:   0%|          | 0/79 [00:00<?, ?it/s]


Epoch 10
Training loss: 0.10309194700344454
Validation loss: 0.5866777958082301
F1 score(weighted): 0.8444446443867011


# Loading(optional) and evaluating model

In [49]:
# model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
#                                                       num_labels=len(label_dict),
#                                                       output_attentions=False,
#                                                       output_hidden_states=False)

In [50]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# model.load_state_dict(
#     torch.load('Models/finetuned_bert_epoch_1_gpu_trained.model',
#                map_location=torch.device('cpu')))

In [51]:
_, predictions, true_vals = evaluate(dataloader_val)

In [54]:
accuracy_per_class(predictions, true_vals)

Class:happy
Accuracy:167/171

Class:not-relevant
Accuracy:17/32

Class:angry
Accuracy:8/9

Class:disgust
Accuracy:0/1

Class:sad
Accuracy:0/5

Class:surprise
Accuracy:1/5

